In [1]:
import math

import numpy as np

In [2]:

def calculate_panel_size_in_pixels(altitude, resolution, sensor_size_mm, focal_length_mm,
                                   physical_panel_size):
    """
    Calculate the expected size of an object in pixels based on camera parameters and object physical size.

    Parameters:
        altitude (float): Altitude in meters.
        resolution (tuple): Image resolution (width, height) in pixels.
        sensor_size_mm (float): Sensor diagonal in millimeters.
        focal_length_m (float): Focal length in millimeters.
        physical_panel_size (tuple): Physical size of the object in meters (width, height).

    Returns:
        tuple: Expected width and height of the object in pixels.
    """
    # Convert sensor diagonal to meters
    sensor_diagonal = sensor_size_mm / 1000  # Convert mm to m
    focal_length = focal_length_mm / 1000

    # Calculate horizontal and vertical Field of View (FoV)
    fov_horizontal = 2 * math.atan(
        (sensor_diagonal / (2 * math.sqrt(1 + (resolution[0] / resolution[1]) ** 2))) / focal_length)
    fov_vertical = 2 * math.atan(
        (sensor_diagonal / (2 * math.sqrt(1 + (resolution[1] / resolution[0]) ** 2))) / focal_length)

    # Calculate scale in pixels per meter
    scale_pixels_per_meter = resolution[1] / (altitude * math.tan(fov_vertical / 2))

    # Calculate expected panel size in pixels
    panel_width_pixels = np.intp(physical_panel_size[0] * scale_pixels_per_meter)
    panel_height_pixels = np.intp(physical_panel_size[1] * scale_pixels_per_meter)

    return panel_width_pixels, panel_height_pixels

In [20]:
focal_length_mm = 5.5
resolution = (1456, 1088)
sensor_size_mm = 6.3
sensor_diagonal = sensor_size_mm / 1000
focal_length = focal_length_mm / 1000
fov_horizontal = 2 * math.atan(
        (sensor_diagonal / (2 * math.sqrt(1 + (resolution[0] / resolution[1]) ** 2))) / focal_length)
# tag sizes in meters for a4 to a1:
tags = [(0.21, 0.210),
        (0.297, 0.297),
        (0.420, 0.420),
        (0.594, 0.594)]

In [12]:
for i, tag in enumerate(tags):
    for height in range(1, 100):
        pixels = calculate_panel_size_in_pixels(height, resolution, sensor_size_mm, focal_length_mm, tag)
        if pixels[0] < 20:
            break
    print(f"A{4 - i}:", height, "meters")

A4: 25 meters
A3: 36 meters
A2: 50 meters
A1: 71 meters


https://optitag.io/blogs/news/designing-your-perfect-apriltag

In [37]:
def get_max_detection_distance(panel_size: float, horizontal_fov: float, horizontal_resolution: float, pixel_per_bit = 5):
    t = panel_size
    f = horizontal_fov
    r = horizontal_resolution
    b = bits = 6  # for 25h9
    p = pixel_per_bit  # hyperparameter, lowest is 2
    return t / (2 * math.tan((b * f * p) / (2 * r)))

In [39]:
for i, tag in enumerate(tags):
    print(f"A{4 - i}:", "Optimal:",int(get_max_detection_distance(tag[0],fov_horizontal, resolution[0])), "meters", "Highest:",int(get_max_detection_distance(tag[0],fov_horizontal, resolution[0], 2)), "meters")

A4: Optimal: 15 meters Highest: 38 meters
A3: Optimal: 21 meters Highest: 54 meters
A2: Optimal: 30 meters Highest: 77 meters
A1: Optimal: 43 meters Highest: 109 meters
